## Deploy a Grafana Dashboard  
To track the different stocks on a live dashboard we will use **Grafana**.  <br>
We will use [Grafwiz](https://github.com/v3io/grafwiz) to define and deploy the dashboard directly from this notebook

In [13]:
# nuclio: ignore
import nuclio

In [15]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

In [16]:
%%nuclio config 
kind = "job"
spec.build.Image = "mlrun/ml-models"

%nuclio: setting kind to 'nuclio'
%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'


In [16]:
%nuclio cmd -c pip install git+https://github.com/v3io/grafwiz --upgrade
%nuclio cmd -c pip install v3io_frames

In [18]:
# nuclio: start-code

In [ ]:
from grafwiz import *
import os
import v3io_frames as v3f

In [40]:
def handler(context,streamview_url,readvector_url,rnn_serving_url,v3io_container,stocks_kv,stocks_tsdb,grafana_url):
    
    context.logger.info("Initializing DataSources1")
    context.logger.info(streamview_url)
    context.logger.info(readvector_url)
    
    DataSource(name='iguazio').deploy(grafana_url, use_auth=True)
    DataSource(name='stream-viewer', frames_url=streamview_url).deploy(grafana_url, use_auth=False, overwrite=False)
    DataSource(name='read-vector', frames_url=readvector_url).deploy(grafana_url, use_auth=False, overwrite=False)
    DataSource(name='rnn-serving', frames_url=rnn_serving_url).deploy(grafana_url, use_auth=False, overwrite=False)
    
    dash = Dashboard("stocks", start='now-7d', dataSource='iguazio')
    
    # Add a symbol combo box (template) with data from the stocks table
    dash.template(name="SYMBOL", label="Symbol", query="fields=symbol;table=" + os.getenv('V3IO_USERNAME') + "/stocks/stocks_kv;backend=kv;container=users")

    # Create a table and log viewer in one row
    tbl = Table('Current Stocks Value', span=12).source(table=stocks_kv,fields=['symbol','price', 'volume','last_updated'],container=v3io_container)
    dash.row([tbl])
    
    # Create 2 charts on the second row
    metrics_row = [Graph(metric).series(table=stocks_tsdb, fields=[metric], filter='symbol=="$SYMBOL"',container=v3io_container) for metric in ['price','volume']]
#     metrics_row.append(Graph('sentiment').series(table=stocks_tsdb, fields=['sentiment'], filter='symbol=="$SYMBOL"', container=v3io_container))
    dash.row(metrics_row)
    
    log = Table('Articles Log', dataSource='stream-viewer', span=12)
    dash.row([log])

    log = Table('Vector Log', dataSource='read-vector', span=12)
    dash.row([log])
    
    log = Table('Predictions', dataSource='rnn-serving', span=12)
    dash.row([log])
    
    dash.deploy(grafana_url)
    
    return "Done"

In [10]:
def init_context(context):
    context.logger.info("init context") 

In [11]:
# nuclio: end-code

In [ ]:
# create a test event and invoke the function locally
init_context(context)
event = nuclio.Event(body='')
resp = handler(context,
               "http://default-tenant.app.dev39.lab.iguazeng.com:30039",
               "http://default-tenant.app.dev39.lab.iguazeng.com:32673", 
               "http://default-tenant.app.dev39.lab.iguazeng.com:31269",
               "users",
               os.getenv('V3IO_USERNAME')  + 'stocks/stocks_kv',
               os.getenv('V3IO_USERNAME') + '/stocks/stocks_tsdb', 
               "http://grafana")

In [8]:
from mlrun import code_to_function
import mlrun
import os 
fn = code_to_function('grafana-tryout',
                      handler='handler')
fn.apply(mlrun.auto_mount())

In [ ]:
fn.run(project = "stocks-" + os.getenv('V3IO_USERNAME'),
       params = {"streamview_url": "http://default-tenant.app.dev39.lab.iguazeng.com:30039",
                           "readvector_url" : "http://default-tenant.app.dev39.lab.iguazeng.com:32673",
                           "rnn_serving_url" : "http://default-tenant.app.dev39.lab.iguazeng.com:31269",
                           "v3io_container" : "users",
                           "stocks_kv" : os.getenv('V3IO_USERNAME')  + 'stocks/stocks_kv',
                           "stocks_tsdb" : os.getenv('V3IO_USERNAME') + '/stocks/stocks_tsdb',
                           "grafana_url" : "http://grafana"})

In [34]:
!curl {addr}

Done